In [3]:
spark.conf.set("spark.sql.shuffle.partitions", 1000)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [4]:
from pyspark.ml.clustering import KMeans, KMeansModel, BisectingKMeans, BisectingKMeansModel
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA, PCAModel, ElementwiseProduct
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.functions import col, sum, greatest, expr, count, countDistinct, max, min, variance, bround, avg, lit, udf
from pyspark.sql import Row
import pandas as pd
import scipy.stats as stats
import matplotlib.pylab as plt
from pyspark.ml.linalg import Vectors
from pyspark.ml.linalg import SparseVector, DenseVector

#PARA EL TRATAMIENTO DE OUTLIERS
from pyspark.sql.functions import col, count, when
from functools import reduce
from operator import and_


### Data Importation

In [5]:
dfEM=spark.read.parquet("/analytics/bc1_clustering/tables/dfEM")

In [6]:
#dfBufferOrig=spark.read.parquet("/analytics/bc1_clustering/tables/dfBufferAccYrGrpPart")
#dfBufferOrig=spark.read.parquet("/analytics/bc1_clustering/tables/dfBufferAccYrSubPart2")
# dfBufferOrig=spark.read.parquet("/analytics/bc1_clustering/tables/dfBufferAccYrGrp")
# dfBufferOrig=spark.read.parquet("/analytics/bc1_clustering/tables/dfFinalMatrix11Dic")
# dfBufferOrig=spark.read.parquet("/analytics/bc1_clustering/tables/dfFinalMatrixGrpVal19Dic")
dfBufferOrig=spark.read.parquet("/analytics/bc1_clustering/tables/dfFinalMatrixGrpVal02Ene")

# dfBufferOrig=dfBufferOrig.withColumnRenamed('','NULO')

In [7]:
# dfBufferOrig.persist()

nameList=dfBufferOrig.schema.names[1:]

len(nameList)

727

In [ ]:
dfBufferOrig.count()

In [ ]:
# [dfBuffer, dfTestBuffer] = dfBufferOrig.randomSplit([0.5, 0.5],666)

In [8]:
dfBuffer=dfBufferOrig.where(~dfBufferOrig['ACCOUNTPK'].cast('string').substr(-1,1).isin(['0','1','2','3','4'])).persist()

In [9]:
dfTestBuffer=dfBufferOrig.where(dfBufferOrig['ACCOUNTPK'].cast('string').substr(-1,1).isin(['0','1','2','3','4'])).persist()

In [ ]:
#dfBuffer.write.parquet("/analytics/bc1_clustering/tables/dfTrainBufferBin")

In [ ]:
#dfTestBuffer.write.parquet("/analytics/bc1_clustering/tables/dfTestBufferBin")

In [ ]:
# dfBuffer=spark.read.parquet("/analytics/bc1_clustering/tables/dfTrainBufferBin")
# dfBuffer.persist()
# nameList=dfBuffer.schema.names[1:]

In [ ]:
# dfTestBuffer=spark.read.parquet("/analytics/bc1_clustering/tables/dfTestBufferBin")
# dfTestBuffer.persist()

In [10]:
dfBuffer.count()

1980853

In [11]:
dfTestBuffer.count()

2010927

In [ ]:
#dfTestBuffer.select(variance(dfTestBuffer.columns[2])).show()

In [ ]:
#dfBufferOrig.select(dfBufferOrig['ACCOUNTPK'].cast('string').substr(-1,1).alias('caracter')).groupBy('caracter').count().show()

In [ ]:
# dfTsys=spark.read.parquet("/analytics/bc1_clustering/tables/dfTsysAccYrOriginal")

In [ ]:
# dfTsys.persist()

In [12]:
dfTsys=dfBufferOrig.select('ACCOUNTPK','Emprestimo_Pessoal','Pagamento_de_Contas','Parcelamento_Fatura_Automatico','Parcelamento_Fatura_Facil','Saque_Parcelado','Saque_a_Vista','Overlimit')

### Binary matrix creation

In [ ]:
dfBufferBinary=dfBufferOrig.select([ i if i=='ACCOUNTPK' else when(col(i)!=0,1).otherwise(0).alias(i) for i in dfBufferOrig.columns])

In [ ]:
dfBufferBinary.persist().count()

In [ ]:
dfBufferBinary.write.parquet("/analytics/bc1_clustering/tables/dfBufferAccYrSubPartBin2")

In [ ]:
# #TEST CONVERSOR A MATRIZ 0/1
# l = [('A',2.0,-3.0,0,-30.0),('B',0,-300.0,1,0),('C',7.0,0.0,2,-50.0),('D',12.0,-700.0,0,0)]
# rdd = sc.parallelize(l)
# dummy = rdd.map(lambda x: Row(ID=x[0], col1=float(x[1]),col2=float(x[2]),col3=float(x[3]),col4=float(x[4])))
# dummyDf = sqlContext.createDataFrame(dummy)
# dummyDf.show(5)
# dummyDf.select([ i if i=='ID' else when(col(i)!=0,1).otherwise(0).alias(i) for i in dummyDf.columns]).show()

### Variable distribution histogram

In [ ]:
#METODO HISTOGRAM DF
plt.clf()
#bins, counts = dummyDf.select('col1').rdd.flatMap(lambda x: x).histogram(20)
bins, counts = dfBuffer.select('CREDITLIMIT').rdd.flatMap(lambda x: x).histogram(20)
plt.hist(bins[:-1], bins=bins, weights=counts)

plt.show()
#.filter(col('101054')>0)

In [ ]:
# #METODO hist DE PLT, parece que no funciona con grandes cantidades de datos
# plt.clf()
# #n, bins, patches = plt.hist(dummyDf.select('col4').collect(), 15, normed=1, facecolor='green', alpha=0.75)
# n, bins, patches = plt.hist(dfBuffer.select('1010540800').collect(), 15, normed=1, facecolor='green', alpha=0.75)
# plt.show()

### Outlier treatment (for money value matrices)

In [ ]:
len(nameList)

In [ ]:
listaCols=[]
listaLimit=[]

for i in nameList:
    cuenta=dfBuffer.filter(col(i)>0).count()
    if cuenta>0:
        listaCols.append(i)
        Q1, Q3 = dfBuffer.filter(col(i)>0).approxQuantile(col=i,probabilities=[0.25,0.75],relativeError=0.01)
        IQR=(Q3-Q1)
        top_limit=Q3+9*IQR
        listaLimit.append(top_limit)

In [ ]:
expr=reduce(and_, [col(c) < t for c, t in zip(listaCols, listaLimit)])

In [ ]:
dfBufferOutlier=dfBuffer.where(expr)

In [ ]:
dfBufferOutlier.persist().count()

In [ ]:
dfBufferOutlier.write.parquet("/analytics/bc1_clustering/tables/dfBufferOutlier")

In [ ]:
dfBufferOutlier=spark.read.parquet("/analytics/bc1_clustering/tables/dfBufferOutlier")

In [ ]:
dfBufferOutlier.count()

### Feature Assembler and Scaler

#### Binary matrices

In [ ]:
dfBufferOutlier=dfBuffer
#dfBufferOutlier=dfTestBuffer.select(dfTestBuffer.columns[0:2400])

In [ ]:
#continuous_feature_assembler= VectorAssembler(inputCols=dfTestBuffer.columns[0:2400], outputCol="features")
continuous_feature_assembler= VectorAssembler(inputCols=nameList, outputCol="features")

In [ ]:
estimatorsKmeans = [continuous_feature_assembler]

featurePipeline = Pipeline(stages=estimatorsKmeans)

sparkFeaturePipelineModel = featurePipeline.fit(dfBufferOutlier)

In [ ]:
dfBufferFeature=sparkFeaturePipelineModel.transform(dfBufferOutlier).persist()

In [ ]:
dfBufferFeature.count()

#### Money value matrices

In [13]:
#SI NO SE EJECUTA EL PROCESO DE OUTLIERS
dfBufferOutlier=dfBuffer
#dfBufferOutlier=dfTestBuffer.select(dfTestBuffer.columns[0:2400])

In [14]:
nameList=dfBuffer.schema.names[1:]

In [15]:
continuous_feature_assembler= VectorAssembler(inputCols=nameList, outputCol="unscaled_continuous_features")

In [16]:
continuous_feature_scaler = StandardScaler(inputCol="unscaled_continuous_features", outputCol="scaled_features",\
                                           withStd=True, withMean=True)

In [17]:
estimatorsKmeans = [continuous_feature_assembler, continuous_feature_scaler]

featurePipeline = Pipeline(stages=estimatorsKmeans)

sparkFeaturePipelineModel = featurePipeline.fit(dfBufferOutlier)

In [18]:
dfBufferScaled=sparkFeaturePipelineModel.transform(dfBufferOutlier).persist()

In [19]:
dfBufferScaled.count()

1980853

In [20]:
dfBufferScaled.printSchema()

root
 |-- ACCOUNTPK: long (nullable = true)
 |-- BALANCE: double (nullable = true)
 |-- INSTALMENTBALANCEFOROVLMCHECK: double (nullable = true)
 |-- CREDITLIMIT: double (nullable = true)
 |-- LimitUse: double (nullable = true)
 |-- LimitOffUs: double (nullable = true)
 |-- LimitPersLoan: double (nullable = true)
 |-- LimitCashOut: double (nullable = true)
 |-- CLOSING_BALANCE: double (nullable = true)
 |-- INSTALLMENTSOFFERS: long (nullable = true)
 |-- Emprestimo_Pessoal: long (nullable = true)
 |-- Pagamento_de_Contas: long (nullable = true)
 |-- Parcelamento_Fatura_Automatico: long (nullable = true)
 |-- Parcelamento_Fatura_Facil: long (nullable = true)
 |-- Saque_Parcelado: long (nullable = true)
 |-- Saque_a_Vista: long (nullable = true)
 |-- Overlimit: long (nullable = true)
 |-- Overlimit_Auth: integer (nullable = true)
 |-- NULO: double (nullable = true)
 |-- 0742: double (nullable = true)
 |-- 0763: double (nullable = true)
 |-- 0780: double (nullable = true)
 |-- 1520: double

#### Increase of the imporance of the financial variables (money)

In [24]:
listaPesos=([1]*727)
#listaPesos[0:17]=[10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10]
VectorPesos=DenseVector(listaPesos)

In [25]:
transformer = ElementwiseProduct(scalingVec=VectorPesos,
                                 inputCol="scaled_features", outputCol="features")
# Batch transform the vectors to create new column:
dfBufferFeature=transformer.transform(dfBufferScaled)

In [26]:
dfBufferFeature.select('scaled_features','features').show(1,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Dimensionality reduction (WIP)

In [ ]:
pca = PCA(k=120, inputCol="features", outputCol="pca_features")

In [ ]:
pcamodel = pca.fit(dfBufferFeature)

In [ ]:
variance=pcamodel.explainedVariance

In [ ]:
variance

In [ ]:
variance.sum()

In [ ]:
dfBufferFeaturePCA=pcamodel.transform(dfBufferFeature)

### K Evaluation

In [ ]:
listaErrores=[]

for k in range(2,51):
    #print(k)
    kmeans = KMeans().setK(k).setSeed(666)
    #kmeans = BisectingKMeans().setK(k).setSeed(666)
    model = kmeans.fit(dfBufferFeature)
    wssse = model.computeCost(dfBufferFeature)
    print("Within Set Sum of Squared Errors {} = ".format(k) + str(wssse))
    listaErrores.append((k,wssse))

In [ ]:
pdErrors=pd.DataFrame.from_records(listaErrores, columns=['clusters','wssse'])

In [ ]:
plt.clf()
plt.plot(pdErrors.clusters,pdErrors.wssse)
plt.show()

In [ ]:
dfErrors=spark.createDataFrame(pdErrors)

In [ ]:
dfErrors.printSchema()

In [ ]:
# dfErrors.write.parquet("/analytics/bc1_clustering/tables/dfErrorsBin")

### Segmentation final model

In [27]:
#kmeans = KMeans().setK(16).setSeed(666)
kmeans = BisectingKMeans().setK(20).setMinDivisibleClusterSize(40000).setSeed(666)

In [28]:
dfBufferFeature.select("ACCOUNTPK","features").show(5)

+---------+--------------------+
|ACCOUNTPK|            features|
+---------+--------------------+
|    67805|[0.53609270966588...|
|   109366|[0.53609270966588...|
|   140126|[-3.4885453062219...|
|   142317|[-0.1599475963050...|
|   164347|[0.53609270966588...|
+---------+--------------------+
only showing top 5 rows



In [29]:
model = kmeans.fit(dfBufferFeature)

### Financial Product Inclination

#### Train

In [30]:
dfTrainRes=model.transform(dfBufferFeature).select(col("ACCOUNTPK").alias("ACCOUNT"),"prediction")

In [31]:
dfProdIncl=dfTrainRes.join(dfBuffer,dfTrainRes.ACCOUNT==dfTsys.ACCOUNTPK,'left_outer').fillna(0)

In [32]:
dfProdIncl.printSchema()

root
 |-- ACCOUNT: long (nullable = true)
 |-- prediction: integer (nullable = true)
 |-- ACCOUNTPK: long (nullable = true)
 |-- BALANCE: double (nullable = false)
 |-- INSTALMENTBALANCEFOROVLMCHECK: double (nullable = false)
 |-- CREDITLIMIT: double (nullable = false)
 |-- LimitUse: double (nullable = false)
 |-- LimitOffUs: double (nullable = false)
 |-- LimitPersLoan: double (nullable = false)
 |-- LimitCashOut: double (nullable = false)
 |-- CLOSING_BALANCE: double (nullable = false)
 |-- INSTALLMENTSOFFERS: long (nullable = true)
 |-- Emprestimo_Pessoal: long (nullable = true)
 |-- Pagamento_de_Contas: long (nullable = true)
 |-- Parcelamento_Fatura_Automatico: long (nullable = true)
 |-- Parcelamento_Fatura_Facil: long (nullable = true)
 |-- Saque_Parcelado: long (nullable = true)
 |-- Saque_a_Vista: long (nullable = true)
 |-- Overlimit: long (nullable = true)
 |-- Overlimit_Auth: integer (nullable = true)
 |-- NULO: double (nullable = false)
 |-- 0742: double (nullable = false)

In [33]:
dfProdIncl.createOrReplaceTempView("tProdIncl")

In [34]:
dfClusIncl=spark.sql("""SELECT prediction, 
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) Overlimit
    from tProdIncl group by prediction order by prediction""")

In [35]:
dfClusIncl.toPandas()

,prediction,Emprestimo_Pessoal,Pagamento_de_Contas,Parcelamento_Fatura_Automatico,Parcelamento_Fatura_Facil,Saque_Parcelado,Saque_a_Vista,Overlimit
0,0,0.010722,0.051376,0.000000,0.339708,0.038080,0.049483,0.436701
1,1,0.006467,0.055442,0.999985,0.361172,0.034396,0.042485,0.342809
2,2,0.016016,0.025781,0.039292,0.114412,0.023532,0.015570,0.153117
3,3,0.000103,0.002410,0.016918,0.022962,0.000054,0.000079,0.072893
4,4,0.000129,0.006941,0.015809,0.057718,0.000731,0.001164,0.098916
5,5,0.001532,0.006600,0.009582,0.080846,0.004036,0.004984,0.043803
6,6,0.001926,0.006056,0.004546,0.035528,0.002773,0.002992,0.028479
7,7,0.006905,0.040064,0.112430,0.141607,0.016004,0.019691,0.191462
8,8,0.006981,0.038111,0.135847,0.165374,0.017530,0.017996,0.277692
9,9,0.006215,0.036390,0.118928,0.130097,0.014112,0.018555,0.204227


In [36]:
dfTrainRes.select("ACCOUNT","prediction").groupBy("prediction").count().orderBy("prediction").toPandas()

,prediction,count
0,0,306931
1,1,130046
2,2,120940
3,3,203338
4,4,154596
5,5,452414
6,6,137047
7,7,108476
8,8,19338
9,9,33306


In [37]:
dfImportanceCluster=spark.sql("""
    SELECT prediction, 
    Emprestimo_Pessoal/AVG_Emprestimo_Pessoal AS Emprestimo_Pessoal, 
    Pagamento_de_Contas/AVG_Pagamento_de_Contas AS Pagamento_de_Contas,
    Parcelamento_Fatura_Automatico/AVG_Parcelamento_Fatura_Automatico AS Parcelamento_Fatura_Automatico,
    Parcelamento_Fatura_Facil/AVG_Parcelamento_Fatura_Facil AS Parcelamento_Fatura_Facil,
    Saque_Parcelado/AVG_Saque_Parcelado  AS Saque_Parcelado,
    Saque_a_Vista/AVG_Saque_a_Vista  AS Saque_a_Vista,
    Overlimit/AVG_Overlimit  AS Overlimit
    FROM
    (SELECT prediction, 
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) Overlimit
    from tProdIncl group by prediction) CLUSTER CROSS JOIN
    (SELECT  
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Overlimit
    from tProdIncl)
    WHERE 
    order by prediction""")

In [38]:
dfImportanceCluster.columns[1:]

['Emprestimo_Pessoal',
 'Pagamento_de_Contas',
 'Parcelamento_Fatura_Automatico',
 'Parcelamento_Fatura_Facil',
 'Saque_Parcelado',
 'Saque_a_Vista',
 'Overlimit']

In [39]:
cond = "psf.when" + ".when".join(["(psf.col('" + c + "') == psf.col('max_value'), psf.lit('" + c + "'))" for c in dfImportanceCluster.columns[1:]])
import pyspark.sql.functions as psf
dfImportanceCluster.withColumn("max_value", psf.greatest(*dfImportanceCluster.columns[1:]))\
    .withColumn("MAX", eval(cond))\
    .toPandas()

,prediction,Emprestimo_Pessoal,Pagamento_de_Contas,Parcelamento_Fatura_Automatico,Parcelamento_Fatura_Facil,Saque_Parcelado,Saque_a_Vista,Overlimit,max_value,MAX
0,0,1.709810,1.711324,0.000000,2.263822,2.071834,2.227258,2.271716,2.271716,Overlimit
1,1,1.031240,1.846746,9.578770,2.406858,1.871360,1.912256,1.783294,9.578770,Parcelamento_Fatura_Automatico
2,2,2.553997,0.858766,0.376377,0.762444,1.280325,0.700795,0.796516,2.553997,Emprestimo_Pessoal
3,3,0.016469,0.080269,0.162053,0.153018,0.002943,0.003542,0.379191,0.379191,Overlimit
4,4,0.020630,0.231191,0.151433,0.384635,0.039768,0.052406,0.514560,0.514560,Overlimit
5,5,0.244263,0.219848,0.091785,0.538761,0.219594,0.224348,0.227862,0.538761,Parcelamento_Fatura_Facil
6,6,0.307182,0.201733,0.043545,0.236759,0.150858,0.134656,0.148149,0.307182,Emprestimo_Pessoal
7,7,1.101055,1.334520,1.076961,0.943674,0.870706,0.886295,0.995983,1.334520,Pagamento_de_Contas
8,8,1.113225,1.269477,1.301263,1.102055,0.953770,0.809988,1.444551,1.444551,Overlimit
9,9,0.991080,1.212130,1.139197,0.866967,0.767769,0.835174,1.062391,1.212130,Pagamento_de_Contas


In [40]:
#dfTrainRes.write.parquet("/analytics/bc1_clustering/tables/dfTrainRes")

In [41]:
#dfTrainRes=spark.read.parquet("/analytics/bc1_clustering/tables/dfTrainRes")
#dfTrainRes.persist()

#### Test

In [42]:
dfTestBufferFeatureTmp=sparkFeaturePipelineModel.transform(dfTestBuffer).persist()

In [43]:
dfTestBufferFeature=transformer.transform(dfTestBufferFeatureTmp)

In [44]:
dfTestRes=model.transform(dfTestBufferFeature).select(col("ACCOUNTPK").alias("ACCOUNT"),"prediction")

In [45]:
dfTestProdIncl=dfTestRes.join(dfTsys,dfTestRes.ACCOUNT==dfTsys.ACCOUNTPK,'left_outer').fillna(0)

In [46]:
dfTestProdIncl.printSchema()

root
 |-- ACCOUNT: long (nullable = true)
 |-- prediction: integer (nullable = true)
 |-- ACCOUNTPK: long (nullable = true)
 |-- Emprestimo_Pessoal: long (nullable = true)
 |-- Pagamento_de_Contas: long (nullable = true)
 |-- Parcelamento_Fatura_Automatico: long (nullable = true)
 |-- Parcelamento_Fatura_Facil: long (nullable = true)
 |-- Saque_Parcelado: long (nullable = true)
 |-- Saque_a_Vista: long (nullable = true)
 |-- Overlimit: long (nullable = true)



In [47]:
dfTestProdIncl.createOrReplaceTempView("tTestProdIncl")

In [48]:
dfTestClusIncl=spark.sql("""SELECT prediction, 
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) Overlimit
    from tTestProdIncl group by prediction order by prediction""")

In [49]:
dfTestClusIncl.toPandas()

,prediction,Emprestimo_Pessoal,Pagamento_de_Contas,Parcelamento_Fatura_Automatico,Parcelamento_Fatura_Facil,Saque_Parcelado,Saque_a_Vista,Overlimit
0,0,0.011434,0.052321,0.000000,0.340114,0.038926,0.050020,0.430798
1,1,0.006878,0.054122,0.999947,0.361137,0.034329,0.042774,0.340556
2,2,0.015854,0.024167,0.040317,0.114994,0.024143,0.016133,0.153144
3,3,0.000152,0.002387,0.017151,0.022862,0.000078,0.000123,0.072972
4,4,0.000168,0.006658,0.015974,0.058276,0.000806,0.001123,0.099746
5,5,0.001626,0.006491,0.009979,0.082300,0.003798,0.005155,0.043533
6,6,0.001986,0.006244,0.004516,0.036704,0.002961,0.003011,0.028639
7,7,0.006677,0.040937,0.111321,0.142961,0.016293,0.019477,0.192440
8,8,0.006991,0.038984,0.133534,0.157975,0.018012,0.018063,0.279824
9,9,0.005988,0.036197,0.114254,0.131566,0.015386,0.018321,0.203990


In [50]:
dfTestRes.select("ACCOUNT","prediction").groupBy("prediction").count().orderBy("prediction").toPandas()

,prediction,count
0,0,312492
1,1,133329
2,2,121859
3,3,204010
4,4,155003
5,5,461859
6,6,139494
7,7,109925
8,8,19598
9,9,33732


#### Total

In [51]:
dfTotal=dfTrainRes.union(dfTestRes)

In [52]:
dfTotalProdIncl=dfTotal.join(dfTsys,dfTrainRes.ACCOUNT==dfTsys.ACCOUNTPK,'left_outer').fillna(0)

In [53]:
dfTotalProdIncl.printSchema()

root
 |-- ACCOUNT: long (nullable = true)
 |-- prediction: integer (nullable = true)
 |-- ACCOUNTPK: long (nullable = true)
 |-- Emprestimo_Pessoal: long (nullable = true)
 |-- Pagamento_de_Contas: long (nullable = true)
 |-- Parcelamento_Fatura_Automatico: long (nullable = true)
 |-- Parcelamento_Fatura_Facil: long (nullable = true)
 |-- Saque_Parcelado: long (nullable = true)
 |-- Saque_a_Vista: long (nullable = true)
 |-- Overlimit: long (nullable = true)



In [54]:
dfTotalProdIncl.createOrReplaceTempView("tTotalProdIncl")

In [55]:
dfTotalClusIncl=spark.sql("""SELECT prediction, 
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) Overlimit
    from tTotalProdIncl group by prediction order by prediction""")

In [56]:
dfTotalClusIncl.toPandas()

,prediction,Emprestimo_Pessoal,Pagamento_de_Contas,Parcelamento_Fatura_Automatico,Parcelamento_Fatura_Facil,Saque_Parcelado,Saque_a_Vista,Overlimit
0,0,0.011081,0.051853,0.000000,0.339913,0.038507,0.049754,0.433723
1,1,0.006675,0.054774,0.999966,0.361154,0.034362,0.042631,0.341669
2,2,0.015935,0.024971,0.039807,0.114704,0.023839,0.015853,0.153131
3,3,0.000128,0.002398,0.017035,0.022912,0.000066,0.000101,0.072933
4,4,0.000149,0.006799,0.015892,0.057998,0.000769,0.001143,0.099332
5,5,0.001579,0.006545,0.009783,0.081581,0.003916,0.005071,0.043666
6,6,0.001956,0.006151,0.004531,0.036121,0.002868,0.003001,0.028560
7,7,0.006790,0.040503,0.111872,0.142289,0.016149,0.019583,0.191954
8,8,0.006986,0.038550,0.134683,0.161650,0.017773,0.018030,0.278765
9,9,0.006101,0.036293,0.116576,0.130836,0.014753,0.018437,0.204108


In [57]:
dfTotalClusIncl.createOrReplaceTempView("tTotalClusIncl")

In [58]:
dfOrden=spark.sql("""SELECT prediction,Emprestimo_Pessoal, 
    row_number() over (order by Emprestimo_Pessoal desc) as orden
    from  tTotalClusIncl order by Emprestimo_Pessoal desc""")

In [59]:
dfOrden.toPandas()

,prediction,Emprestimo_Pessoal,orden
0,11,0.019818,1
1,12,0.018392,2
2,2,0.015935,3
3,15,0.014794,4
4,14,0.014782,5
5,16,0.014729,6
6,13,0.014706,7
7,0,0.011081,8
8,19,0.009623,9
9,18,0.008543,10


In [60]:
dfTotal.select("ACCOUNT","prediction").groupBy("prediction").count().orderBy("prediction") \
    .withColumn('Percentage', col('count')/dfTotal.count()).toPandas()

,prediction,count,Percentage
0,0,619423,0.155175
1,1,263375,0.065979
2,2,242799,0.060825
3,3,407348,0.102047
4,4,309599,0.077559
5,5,914273,0.229039
6,6,276541,0.069278
7,7,218401,0.054713
8,8,38936,0.009754
9,9,67038,0.016794


In [61]:
dfTotalImportanceCluster=spark.sql("""
    SELECT prediction, 
    Emprestimo_Pessoal/AVG_Emprestimo_Pessoal AS Emprestimo_Pessoal, 
    Pagamento_de_Contas/AVG_Pagamento_de_Contas AS Pagamento_de_Contas,
    Parcelamento_Fatura_Automatico/AVG_Parcelamento_Fatura_Automatico AS Parcelamento_Fatura_Automatico,
    Parcelamento_Fatura_Facil/AVG_Parcelamento_Fatura_Facil AS Parcelamento_Fatura_Facil,
    Saque_Parcelado/AVG_Saque_Parcelado  AS Saque_Parcelado,
    Saque_a_Vista/AVG_Saque_a_Vista  AS Saque_a_Vista,
    Overlimit/AVG_Overlimit  AS Overlimit
    FROM
    (SELECT prediction, 
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) Overlimit
    from tTotalProdIncl group by prediction) CLUSTER CROSS JOIN
    (SELECT  
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Overlimit
    from tTotalProdIncl)
    WHERE 
    order by prediction""")

In [62]:
dfOverallPop=spark.sql("""SELECT  
    SUM(CASE WHEN Emprestimo_Pessoal>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Emprestimo_Pessoal,
    SUM(CASE WHEN Pagamento_de_Contas>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Pagamento_de_Contas, 
    SUM(CASE WHEN Parcelamento_Fatura_Automatico>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Parcelamento_Fatura_Automatico, 
    SUM(CASE WHEN Parcelamento_Fatura_Facil>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Parcelamento_Fatura_Facil, 
    SUM(CASE WHEN Saque_Parcelado>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Saque_Parcelado, 
    SUM(CASE WHEN Saque_a_Vista>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Saque_a_Vista,
    SUM(CASE WHEN Overlimit>0 THEN 1 ELSE 0 END)/SUM(1) AVG_Overlimit
    from tTotalProdIncl
    """)

In [63]:
dfOverallPop.toPandas()

,AVG_Emprestimo_Pessoal,AVG_Pagamento_de_Contas,AVG_Parcelamento_Fatura_Automatico,AVG_Parcelamento_Fatura_Facil,AVG_Saque_Parcelado,AVG_Saque_a_Vista,AVG_Overlimit
0,0.006323,0.030089,0.104702,0.150508,0.018516,0.022339,0.191878


In [64]:
dfTotalImportanceCluster.columns[1:]

['Emprestimo_Pessoal',
 'Pagamento_de_Contas',
 'Parcelamento_Fatura_Automatico',
 'Parcelamento_Fatura_Facil',
 'Saque_Parcelado',
 'Saque_a_Vista',
 'Overlimit']

In [65]:
cond = "psf.when" + ".when".join(["(psf.col('" + c + "') == psf.col('max_value'), psf.lit('" + c + "'))" for c in dfTotalImportanceCluster.columns[1:]])
import pyspark.sql.functions as psf
dfTotalImportanceCluster.withColumn("max_value", psf.greatest(*dfTotalImportanceCluster.columns[1:]))\
    .withColumn("MAX", eval(cond))\
    .toPandas()

,prediction,Emprestimo_Pessoal,Pagamento_de_Contas,Parcelamento_Fatura_Automatico,Parcelamento_Fatura_Facil,Saque_Parcelado,Saque_a_Vista,Overlimit,max_value,MAX
0,0,1.752398,1.723305,0.000000,2.258442,2.079645,2.227227,2.260415,2.260415,Overlimit
1,1,1.055570,1.820366,9.550575,2.399572,1.855777,1.908363,1.780660,9.550575,Parcelamento_Fatura_Automatico
2,2,2.519966,0.829904,0.380189,0.762113,1.287459,0.709634,0.798065,2.519966,Emprestimo_Pessoal
3,3,0.020187,0.079711,0.162696,0.152229,0.003580,0.004506,0.380100,0.380100,Overlimit
4,4,0.023496,0.225964,0.151778,0.385346,0.041517,0.051184,0.517683,0.517683,Overlimit
5,5,0.249766,0.217522,0.093433,0.542036,0.211475,0.226987,0.227574,0.542036,Parcelamento_Fatura_Facil
6,6,0.309372,0.204424,0.043275,0.239996,0.154869,0.134354,0.148845,0.309372,Emprestimo_Pessoal
7,7,1.073815,1.346107,1.068480,0.945391,0.872173,0.876633,1.000400,1.346107,Pagamento_de_Contas
8,8,1.104741,1.281200,1.286340,1.074030,0.959857,0.807084,1.452828,1.452828,Overlimit
9,9,0.964817,1.206170,1.113403,0.869299,0.796759,0.825336,1.063741,1.206170,Pagamento_de_Contas


### Cluster Analysis

#### KS2 Evaluation

In [154]:
dfOrden.persist().toPandas()

,prediction,Emprestimo_Pessoal,orden
0,11,0.019818,1
1,12,0.018392,2
2,2,0.015935,3
3,15,0.014794,4
4,14,0.014782,5
5,16,0.014729,6
6,13,0.014706,7
7,0,0.011081,8
8,19,0.009623,9
9,18,0.008543,10


In [73]:
dfValidacionNov=spark.read.parquet("/analytics/bc1_clustering/tables/dfValidacionNov")

In [76]:
dfValidacionNov.printSchema()

root
 |-- TRANSACTIONSERNO: long (nullable = true)
 |-- ACCOUNTSERNOFK: long (nullable = true)
 |-- TYPESERNO_GLEDGER: long (nullable = true)
 |-- VALUEDATE: string (nullable = true)
 |-- I013_TRXN_DATE: string (nullable = true)
 |-- POSTDATE: string (nullable = true)
 |-- STGENERAL: string (nullable = true)
 |-- REFDATE: timestamp (nullable = true)
 |-- FG_CURRENT: integer (nullable = true)
 |-- TM_INSERT: timestamp (nullable = true)
 |-- PRODUTO: string (nullable = true)



In [75]:
dfValidacionNov.where(col('PRODUTO').like('Empr%')).select('ACCOUNTSERNOFK')

DataFrame[ACCOUNTSERNOFK: bigint]

In [88]:
dfEvalTmp1=dfTotalProdIncl.select('ACCOUNT','prediction','Emprestimo_Pessoal') \
    .join(dfValidacionNov.where(col('PRODUTO').like('Empr%')).select('ACCOUNTSERNOFK'),dfValidacionNov.ACCOUNTSERNOFK==dfTotalProdIncl.ACCOUNT,'left_outer') \
    .withColumn('EP',when(col('ACCOUNTSERNOFK').isNull(),0).otherwise(1)) 

In [89]:
dfEvalTmp1.persist()

+-------+----------+------------------+--------------+---+
|ACCOUNT|prediction|Emprestimo_Pessoal|ACCOUNTSERNOFK| EP|
+-------+----------+------------------+--------------+---+
|  13401|         2|                 0|          null|  0|
|  19141|         0|                 0|          null|  0|
| 102561|        12|                 0|          null|  0|
| 111609|         7|                 0|          null|  0|
| 116724|         2|                 0|          null|  0|
| 120327|        10|                 0|          null|  0|
| 122127|        14|                 0|          null|  0|
| 129421|         1|                 0|          null|  0|
| 132353|        14|                 0|          null|  0|
| 137489|         8|                 0|          null|  0|
| 144245|        11|                 0|          null|  0|
| 149806|         6|                 0|          null|  0|
| 157951|         5|                 0|          null|  0|
| 166887|         1|                 0|          null|  

In [91]:
dfEvalTmp1.groupBy('EP','Emprestimo_Pessoal').count().show()

+---+------------------+-------+
| EP|Emprestimo_Pessoal|  count|
+---+------------------+-------+
|  1|                 1|     23|
|  0|                 0|3965535|
|  1|                 0|   1003|
|  0|                 1|  25219|
+---+------------------+-------+



In [97]:
dfEvalTmp1.printSchema()

root
 |-- ACCOUNT: long (nullable = true)
 |-- prediction: integer (nullable = true)
 |-- Emprestimo_Pessoal: long (nullable = true)
 |-- ACCOUNTSERNOFK: long (nullable = true)
 |-- EP: integer (nullable = false)



In [133]:
dfEval=dfEvalTmp1.join(dfOrden,dfEvalTmp1.prediction==dfOrden.prediction).drop(dfOrden.prediction).drop(dfOrden.Emprestimo_Pessoal) \
    .drop(dfEvalTmp1.ACCOUNTSERNOFK)

In [138]:
dfEval=dfEvalTmp1.join(dfOrden,dfEvalTmp1.prediction==dfOrden.prediction) \
    .select('ACCOUNT',dfEvalTmp1.prediction, 'EP','orden')

In [139]:
dfEval.printSchema()

root
 |-- ACCOUNT: long (nullable = true)
 |-- prediction: integer (nullable = true)
 |-- EP: integer (nullable = false)
 |-- orden: integer (nullable = true)



In [127]:
Positives=[i.orden for i in dfEval.where(col('EP')==1).select('orden').collect()]

In [129]:
Negatives=[i.orden for i in dfEval.where(col('EP')==0).select('orden').collect()]

In [130]:
stats.ks_2samp(Positives,Negatives)

Ks_2sampResult(statistic=0.44572956802905273, pvalue=8.4331791957173511e-179)

In [153]:
dfEval.groupBy('prediction').agg((sum('EP')/count('*')).alias('porcentaje')).orderBy('porcentaje',ascending=False).toPandas()

,prediction,porcentaje
0,14,0.001176
1,11,0.001172
2,15,0.000971
3,13,0.000937
4,16,0.000722
5,19,0.000671
6,2,0.000671
7,12,0.000563
8,0,0.000373
9,17,0.000359


#### Variable analysis

In [ ]:
dfAnVar=dfBufferOrig.join(dfTotal,dfTotal.ACCOUNT==dfBufferOrig.ACCOUNTPK).drop(dfTotal.ACCOUNT).persist()

In [ ]:
dfAnVar.printSchema()

In [ ]:
dfAnVar.select('BALANCE').collect()

In [ ]:
total=dfAnVar.select('BALANCE').collect()

In [ ]:
cluster=dfAnVar.where(col('prediction')==4).select('BALANCE').collect()

In [ ]:
stats.f_oneway(total,cluster)

In [ ]:
dfTrainRes.printSchema()

#### Products

In [ ]:
#dfAnalysis=spark.read.parquet("/analytics/bc1_clustering/tables/dfAnalysisBin")
dfAnalysis=spark.read.parquet("/analytics/bc1_clustering/tables/dfAnalysis11Dic")

In [ ]:
dfAnalysis.printSchema()

In [ ]:
dfAnTmp1=dfAnalysis.join(dfTrainRes,dfTrainRes.ACCOUNT==dfAnalysis.ACCOUNTPK)

In [ ]:
dfAnTmp1.printSchema()

In [ ]:
dfAnTmp1.createOrReplaceTempView('tdfAnTmp1')

In [ ]:
dfSectorImportance=spark.sql("""SELECT * FROM (
    SELECT prediction, SECTOR, IMPORTANCE, 
    row_number() over (partition by prediction order by IMPORTANCE desc) as orden FROM 
    (
    SELECT prediction, SECTOR, AVGPRED/AVGTOTAL AS IMPORTANCE FROM 
    (SELECT PRF_DS_SETOR as SECTOR, SUM(PURCHASES)/COUNT(DISTINCT ACCOUNT) AS AVGTOTAL
    FROM tdfAnTmp1 GROUP BY PRF_DS_SETOR ORDER BY SECTOR) A,
    (SELECT prediction, PRF_DS_SETOR as SECTORPRED, SUM(PURCHASES)/COUNT(DISTINCT ACCOUNT) AS AVGPRED
    FROM tdfAnTmp1 GROUP BY prediction, PRF_DS_SETOR ORDER BY SECTORPRED) B
    WHERE SECTOR=SECTORPRED ORDER BY prediction, SECTOR) MAIN
    ORDER BY prediction, orden) FINAL
    WHERE orden=1""")
dfSectorImportance.printSchema()

In [ ]:
dfSectorImportance.toPandas()

#### Demographic

In [ ]:
#LECTURA DE TABLA PARQUET
dfPeople=spark.read.parquet("/analytics/bc1_clustering/tables/dfPeople")
dfPeople=dfPeople.withColumnRenamed("SERNO","PEOPLESERNO").withColumn('YOB',(dfPeople["DOB"]/10000).cast('integer'))
dfAccounts=spark.read.parquet("/analytics/bc1_clustering/tables/dfAccountsRefdate")
dfAccounts=dfAccounts.withColumnRenamed("SERNO","ACCOUNTSERNO").withColumnRenamed("PEOPLESERNO","PEOPLESERNOFK")

In [ ]:
dfDemographic=dfTrainRes.join(dfAccounts,dfAccounts.ACCOUNTSERNO==dfTrainRes.ACCOUNT) \
    .join(dfPeople,dfPeople.PEOPLESERNO==dfAccounts.PEOPLESERNOFK) \
    .select('ACCOUNTSERNO','prediction','PEOPLESERNO','SEX','MARITAL_STATUS','YOB').persist()

In [ ]:
dfDemographic.createOrReplaceTempView("tDemographic")

In [ ]:
dfAge=spark.sql("""SELECT prediction, round(AVG(YOB),0) as cuenta from tDemographic group by prediction order by prediction""")
dfAge.toPandas()

In [ ]:
dfSex=spark.sql("""SELECT A.*, B.total, round(cuenta/total,2) as porcentaje FROM 
    (SELECT prediction, SEX, COUNT(*) as cuenta from tDemographic group by prediction, SEX) A,
    (SELECT prediction, COUNT(*) as total from tDemographic group by prediction) B
    WHERE A.prediction=B.prediction order by prediction, SEX""")
dfSex.toPandas()

In [ ]:
dfMarital=spark.sql("""SELECT A.*, B.total, round(cuenta/total,2) as porcentaje FROM 
    (SELECT prediction, MARITAL_STATUS, COUNT(*) as cuenta from tDemographic group by prediction, MARITAL_STATUS) A,
    (SELECT prediction, COUNT(*) as total from tDemographic group by prediction) B
    WHERE A.prediction=B.prediction order by prediction, MARITAL_STATUS""")
dfMarital.toPandas()

### Model saving

In [ ]:
#Iter2 -> grupo, part, dinero, 6*IQR

In [66]:
model.save("/analytics/bc1_clustering/models/model02EneGrpNWVal")

In [67]:
sparkFeaturePipelineModel.save("/analytics/bc1_clustering/models/pipeline02EneGrpNWVal")

In [68]:
transformer.save("/analytics/bc1_clustering/models/scaleFinancial02EneGrpNWVal")

In [69]:
dfOrden.write.parquet("/analytics/bc1_clustering/tables/dfOrden02EneGrpNWVal")

In [ ]:
dfOrden=spark.read.parquet("/analytics/bc1_clustering/tables/dfOrden02EneGrpNWVal")

In [ ]:
transformer=ElementwiseProduct.load("/analytics/bc1_clustering/models/scaleFinancial02EneGrpNWVal")

In [ ]:
sparkFeaturePipelineModel=PipelineModel.load("/analytics/bc1_clustering/models/pipeline02EneGrpNWVal")

In [ ]:
model=BisectingKMeansModel.load("/analytics/bc1_clustering/models/model02EneGrpNWVal")